In [1]:
dataset_dir = "data_fake_real"
print("Loading dataset from: " + dataset_dir)

Loading dataset from: data_fake_real


In [2]:
import tensorflow as tf

img_height = 32 
img_width = 32
BATCH_SIZE = 32

# Loading training data
train_ds = tf.keras.utils.image_dataset_from_directory(
  dataset_dir + "/train",
  seed=10,
  image_size=(img_height, img_width),
  batch_size=BATCH_SIZE)

# Loading validation data
val_ds = tf.keras.utils.image_dataset_from_directory(
  dataset_dir + "/test",
  seed=10,
  image_size=(img_height, img_width),
  batch_size=BATCH_SIZE)

# Checking if it is loaded properly
print("Training Classes:")
class_names = train_ds.class_names
print(class_names)

print("Testing Classes:")
class_names = val_ds.class_names
print(class_names)

C:\Users\DELL\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\DELL\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


Found 100000 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.
Training Classes:
['FAKE', 'REAL']
Testing Classes:
['FAKE', 'REAL']


In [3]:
#Building the architecture 

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input

model = Sequential()
    
# Block 1
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

# Block 2
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))


# Final Classifier Block
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


# from tensorflow.keras.models import load_model
# model = load_model('9-6-train-0.95-5epoxhs.h5')

model.compile(
  optimizer='adam',
  loss=tf.keras.losses.BinaryCrossentropy(),
  metrics=['accuracy'])

# Build the model so we can see a summary
model.build(input_shape=(None, 32, 32, 3))
model.summary()





Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (Batch  (None, 32, 32, 32)        128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        18496     
                                                                 
 batch_normalization_1 (Bat  (None, 32, 32, 64)        256       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 64)        0         
 D)                                                              
                                                     

In [4]:
# Reshaping x_labels 
(x_train, y_train) = next(iter(train_ds))
(x_val, y_val) = next(iter(val_ds))
tf.experimental.numpy.experimental_enable_numpy_behavior()
x_train_reshaped = x_train.reshape(-1, 32, 32, 3)
x_val_reshaped = x_val.reshape(-1, 32, 32, 3)


In [5]:
print(x_train.shape)
print(y_train.shape)
print(x_train_reshaped.shape)

(32, 32, 32, 3)
(32,)
(32, 32, 32, 3)


In [7]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Defining callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

callbacks_list = [early_stopping, reduce_lr]

# Data augmentation
data_generator = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    shear_range=0.2,
    fill_mode='nearest'

)

train_generator = data_generator.flow(x_train_reshaped, y_train, batch_size=BATCH_SIZE)


print("Starting training...")
history = model.fit(
  train_ds,
  validation_data=(x_val_reshaped, y_val),
  epochs=5,
  callbacks=callbacks_list,
  verbose=1
)
print("Training finished.")

Starting training...
Epoch 1/5
3125/3125 [==============================] - 1823s 583ms/step - loss: 0.2424 - accuracy: 0.9031 - val_loss: 0.2171 - val_accuracy: 0.9375 - lr: 0.0010
Epoch 2/5
3125/3125 [==============================] - 2004s 641ms/step - loss: 0.1796 - accuracy: 0.9301 - val_loss: 0.2709 - val_accuracy: 0.9375 - lr: 0.0010
Epoch 3/5
3125/3125 [==============================] - 2087s 667ms/step - loss: 0.1637 - accuracy: 0.9369 - val_loss: 0.0479 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 4/5
3125/3125 [==============================] - 1822s 583ms/step - loss: 0.1452 - accuracy: 0.9440 - val_loss: 0.1348 - val_accuracy: 0.9375 - lr: 0.0010
Epoch 5/5
3125/3125 [==============================] - 1941s 621ms/step - loss: 0.1298 - accuracy: 0.9505 - val_loss: 0.2182 - val_accuracy: 0.9062 - lr: 0.0010
Training finished.


In [ ]:
model.save('9-6-train-0.95-5epoxhs.h5')

In [8]:
l, a = model.evaluate(val_ds, verbose=1)
print("Test loss: ", l)
print("Test accuracy:", a)

625/625 [==============================] - 163s 259ms/step - loss: 0.1293 - accuracy: 0.9504
Test loss:  0.12930308282375336
Test accuracy: 0.9503999948501587


In [ ]:
model.save('13-06-test-0.93-5epochs.h5')